In [1]:
#export
from zero.imports import *
from zero.core import *

### Folded ResNet

In [2]:
11 // 2, 11 % 2

(5, 1)

In [62]:
#export
class ResNet5(nn.Module):
    def __init__(self, Stem, Unit, ni:int=32, num_stages:int=4, num_units:int=6, exp:int=2,
                bottle_scale:int=1, first_downsample:bool=True, c_in:int=3, c_out:int=10, **kwargs):
        super(ResNet5, self).__init__()
        assert num_units >= 2 * (5-1)
        nhs = [ni * exp ** i for i in range(num_stages)] 
        nos = [nh*bottle_scale for nh in nhs]
        nus = [num_units] * num_stages  # all stages have the same number of units
        strides = [1 if i==0 and not first_downsample else 2 for i in range(num_stages)]
        print('nhs=', nhs, 'nos=', nos, 'nus=', nus, 'strides=', strides)
        
        ni = nhs[0]  # 
        self.stem = Stem(c_in, ni)
        self.unit0 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3, x4, x5
        self.unit1 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3, x4, x5
        self.unit2 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3, x4, x5
        self.unit3 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3, x4, x5
        units = []
        idmappings = []
        cur = 0
        for i, (nh, no, nu, stride) in enumerate(zip(nhs, nos, nus, strides)):
            if cur % 8 == 0 or cur % 8 == 4:  # 22221112
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
            elif cur % 8 == 1 or cur % 8 == 5: # 22211221
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
            elif cur % 8 == 2 or cur % 8 == 6:  # 22122211
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
            elif cur % 8 == 3 or cur % 8 == 7:  # 22222111
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                
            for j in range(nu - 8):
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
            cur += nu
            ni = no
        self.units = nn.ModuleList(units)
        self.idmappings = nn.ModuleList(idmappings)
        
        self.classifier = Classifier(nos[-1], c_out)
        init_cnn(self)
        
    def forward(self, x):
        x1 = self.stem(x)
        x2 = self.unit0(x1) + x1
        x3 = self.unit1(x2) + x2
        x4 = self.unit2(x3) + x3
        x5 = self.unit3(x4) + x4
        for i, (unit, idmapping) in enumerate(zip(self.units, self.idmappings)):
            if i % 8 == 0:
                x4 = unit(x5) + idmapping(x4)
            elif i % 8 == 1:
                x3 = unit(x4) + idmapping(x3)
            elif i % 8 == 2:
                x2 = unit(x3) + idmapping(x2)
            elif i % 8 == 3:
                x1 = unit(x2) + idmapping(x1)
            elif i % 8 == 4:
                x2 = unit(x1) + idmapping(x2)
            elif i % 8 == 5:
                x3 = unit(x2) + idmapping(x3)
            elif i % 8 == 6:
                x4 = unit(x3) + idmapping(x4)
            elif i % 8 == 7:
                x5 = unit(x4) + idmapping(x5)
        x = x3 if i%8==0 or i%8==6 else x3 if i%8==1 or i%8==5 else x2 if i%8==2 or i%8==4 else x1 if i%8==3 else x5

        x = self.classifier(x)
        return x
        

In [75]:
#export
class ResNet4(nn.Module):
    def __init__(self, Stem, Unit, ni:int=32, num_stages:int=4, num_units:int=6, exp:int=2,
                bottle_scale:int=1, first_downsample:bool=True, c_in:int=3, c_out:int=10, **kwargs):
        super(ResNet4, self).__init__()
        assert num_units >= 2 * (4-1)
        nhs = [ni * exp ** i for i in range(num_stages)] 
        nos = [nh*bottle_scale for nh in nhs]
        nus = [num_units] * num_stages  # all stages have the same number of units
        strides = [1 if i==0 and not first_downsample else 2 for i in range(num_stages)]
        print('nhs=', nhs, 'nos=', nos, 'nus=', nus, 'strides=', strides)
        
        ni = nhs[0]  # 
        self.stem = Stem(c_in, ni)
        self.unit0 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3, x4
        self.unit1 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3, x4
        self.unit2 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3, x4
        units = []
        idmappings = []
        cur = 0
        for i, (nh, no, nu, stride) in enumerate(zip(nhs, nos, nus, strides)):
            if cur % 6 == 0 or cur % 6 == 3:
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
            elif cur % 6 == 1 or cur % 6 == 4:
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
            elif cur % 6 == 2 or cur % 6 == 5:
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                
            for j in range(nu - 6):
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
            cur += nu
            ni = no
        self.units = nn.ModuleList(units)
        self.idmappings = nn.ModuleList(idmappings)
        
        self.classifier = Classifier(nos[-1], c_out)
        init_cnn(self)
        
    def forward(self, x):
        x1 = self.stem(x)
        x2 = self.unit0(x1) + x1
        x3 = self.unit1(x2) + x2
        x4 = self.unit2(x3) + x3
        for i, (unit, idmapping) in enumerate(zip(self.units, self.idmappings)):
            if i % 6 == 0:
                x3 = unit(x4) + idmapping(x3)
            elif i % 6 == 1:
                x2 = unit(x3) + idmapping(x2)
            elif i % 6 == 2:
                x1 = unit(x2) + idmapping(x1)
            elif i % 6 == 3:
                x2 = unit(x1) + idmapping(x2)
            elif i % 6 == 4:
                x3 = unit(x2) + idmapping(x3)
            elif i % 6 == 5:
                x4 = unit(x3) + idmapping(x4)
        x = x3 if i % 6 == 0 or i % 6 == 4 else x2 if i % 6 == 1 or i % 6 == 3 else x1 if i % 6 == 2 else x4
        x = self.classifier(x)
        return x
        

In [76]:
#export
class ResNet3(nn.Module):
    def __init__(self, Stem, Unit, ni:int=32, num_stages:int=4, num_units:int=6, exp:int=2,
                bottle_scale:int=1, first_downsample:bool=True, c_in:int=3, c_out:int=10, **kwargs):
        super(ResNet3, self).__init__()
        assert num_units >= 2 * (3-1)
        nhs = [ni * exp ** i for i in range(num_stages)] 
        nos = [nh*bottle_scale for nh in nhs]
        nus = [num_units] * num_stages  # all stages have the same number of units
        strides = [1 if i==0 and not first_downsample else 2 for i in range(num_stages)]
        print('nhs=', nhs, 'nos=', nos, 'nus=', nus, 'strides=', strides)
        
        ni = nhs[0]  # 
        self.stem = Stem(c_in, ni)
        self.unit0 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3
        self.unit1 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2, x3
        units = []
        idmappings = []
        cur = 0
        for i, (nh, no, nu, stride) in enumerate(zip(nhs, nos, nus, strides)):
            if cur % 4 == 0 or cur % 4 == 2:
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
            elif cur % 4 == 1 or cur % 4 == 3:
                units += [Unit(ni, no, nh, stride=stride, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(ni, no, stride=stride)]
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
                
            for j in range(nu - 4):
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
            cur += nu
            ni = no
        self.units = nn.ModuleList(units)
        self.idmappings = nn.ModuleList(idmappings)
        
        self.classifier = Classifier(nos[-1], c_out)
        init_cnn(self)
        
    def forward(self, x):
        x1 = self.stem(x)
        x2 = self.unit0(x1) + x1
        x3 = self.unit1(x2) + x2
        for i, (unit, idmapping) in enumerate(zip(self.units, self.idmappings)):
            if i % 4 == 0:
                x2 = unit(x3) + idmapping(x2)
            elif i % 4 == 1:
                x1 = unit(x2) + idmapping(x1)
            elif i % 4 == 2:
                x2 = unit(x1) + idmapping(x2)
            elif i % 4 == 3:
                x3 = unit(x2) + idmapping(x3)
        x = x2 if i % 4 == 0 or i % 4 == 2 else x1 if i % 4 == 0 else x3
        x = self.classifier(x)
        return x
        

In [77]:
#export
class ResNet2(nn.Module):
    def __init__(self, Stem, Unit, ni:int=32, num_stages:int=4, num_units:int=6, exp:int=2,
                bottle_scale:int=1, first_downsample:bool=True, c_in:int=3, c_out:int=10, **kwargs):
        super(ResNet2, self).__init__()
        nhs = [ni * exp ** i for i in range(num_stages)] 
        nos = [nh*bottle_scale for nh in nhs]
        nus = [num_units] * num_stages  # all stages have the same number of units
        strides = [1 if i==0 and not first_downsample else 2 for i in range(num_stages)]
        print('nhs=', nhs, 'nos=', nos, 'nus=', nus, 'strides=', strides)
        
        ni = nhs[0]  # 
        self.stem = Stem(c_in, ni)
        self.unit0 = Unit(ni, ni, ni, stride=1, **kwargs) # expand to x1, x2
        units = []
        idmappings = []
        for i, (nh, no, nu, stride) in enumerate(zip(nhs, nos, nus, strides)):
            units += [Unit(ni, no, nh, stride=stride, **kwargs)]
            idmappings += [IdentityMapping(ni, no, stride=stride)]
            units += [Unit(no, no, nh, stride=1, **kwargs)]
            idmappings += [IdentityMapping(ni, no, stride=stride)]
                
            for j in range(nu - 2):
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
            ni = no
        self.units = nn.ModuleList(units)
        self.idmappings = nn.ModuleList(idmappings)
        
        self.classifier = Classifier(nos[-1], c_out)
        init_cnn(self)
        
    def forward(self, x):
        x1 = self.stem(x)
        x2 = self.unit0(x1) + x1
        for i, (unit, idmapping) in enumerate(zip(self.units, self.idmappings)):
            if i % 2 == 0:
                x1 = unit(x2) + idmapping(x1)
            elif i % 2 == 1:
                x2 = unit(x1) + idmapping(x2)
        x = x1 if i % 2 == 0 else x2
        x = self.classifier(x)
        return x
        

In [78]:
#export
class ResNet1(nn.Module):
    """
    A folded residual network.

    Parameters:
    -----------
    Stem : class of the stem layer.
    Stage : class of the stages.
    Unit : class of the basic blocks.
    ni  :  number of input channels of the first stage 
            equal to number of output channels of the stem layer.
    num_stages : number of stages.
    num_units : number of units per stage.
    exp : expansion coefficient for number of channels increasing with stages.
    bottle_scale : bottleneck coefficient.
    first_downsample : does downsample at the first stage.
    c_in : number of input channels of the stem layer.
    c_out : 
    """
    def __init__(self, Stem, Unit, ni:int=32, num_stages:int=4, num_units:int=6, exp:int=2,
                bottle_scale:int=1, first_downsample:bool=True, c_in:int=3, c_out:int=10, **kwargs):
        super(ResNet1, self).__init__()
        nhs = [ni * exp ** i for i in range(num_stages)] 
        nos = [nh*bottle_scale for nh in nhs]
        nus = [num_units] * num_stages  # all stages have the same number of units
        strides = [1 if i==0 and not first_downsample else 2 for i in range(num_stages)]
        print('nhs=', nhs, 'nos=', nos, 'nus=', nus, 'strides=', strides)
        ni = nhs[0]
        self.stem = Stem(c_in, ni)
        units = []
        idmappings = []
        for nh, no, nu, stride in zip(nhs, nos, nus, strides):
            units += [Unit(ni, no, nh, stride=stride, **kwargs)]
            idmappings += [IdentityMapping(ni, no, stride=stride)]
            for j in range(nu-1):
                units += [Unit(no, no, nh, stride=1, **kwargs)]
                idmappings += [IdentityMapping(no, no, stride=1)]
            ni = no
        self.units = nn.ModuleList(units)
        self.idmappings = nn.ModuleList(idmappings)
        
        self.classifier = Classifier(nos[-1], c_out)
        init_cnn(self)
        
    def forward(self, x):
        x = self.stem(x)
        for unit, idmapping in zip(self.units, self.idmappings):
            x = unit(x) + idmapping(x)
        x = self.classifier(x)
        return x
        

In [71]:
model = ResNet5(Stem = conv_bn, Unit = xception, ni=32, num_stages=5, num_units=15, 
                exp=1, first_downsample=False, zero_bn=True)

nhs= [32, 32, 32, 32, 32] nos= [32, 32, 32, 32, 32] nus= [15, 15, 15, 15, 15] strides= [1, 2, 2, 2, 2]


In [72]:
num_params(model)

tensor(109962)

In [73]:
x = torch.randn(2,3,64,64)

In [74]:
with torch.autograd.set_detect_anomaly(True):
    out = model(x)
    out.mean().backward()

In [50]:
model

ResNet4(
  (stem): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (unit0): Sequential(
    (0): ReLU()
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (unit1): Sequential(
    (0): ReLU()
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (2): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (unit2): Sequential(
    (0): ReLU()
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (2): Conv2d(32, 32, kernel_size=(1, 1), stride=

### Export: Before Export, First Remove `zero` in `from zero.bla import *`

In [80]:
!python notebook2script.py resnetx.ipynb

Converted resnetx.ipynb to zero/resnetx.py
